In [1]:
"""
ANFIS (Adaptive Neuro-Fuzzy Inference System) Implementation
Prediksi Produksi Jagung Agroindustri

Author: [Rifki Setiawan - G.211.22.0128]
Date: December 2025
Course: UAS Fuzzy Logic - Gasal 2024/2025
"""

import numpy as np


def gaussian_membership(x, mean, sigma):
    """
    Fungsi membership Gaussian untuk ANFIS

    Parameters:
    -----------
    x : float
        Nilai input
    mean : float
        Mean/center dari Gaussian
    sigma : float
        Standard deviation dari Gaussian

    Returns:
    --------
    float : Nilai membership [0, 1]
    """
    return np.exp(-((x - mean) ** 2) / (2 * sigma ** 2))


def anfis_predict(soil_moisture, soil_ph, avg_temp, verbose=True):
    """
    ANFIS untuk memprediksi produksi jagung

    Parameters:
    -----------
    soil_moisture : float
        Kelembaban tanah (%)
    soil_ph : float
        Keasaman tanah (pH)
    avg_temp : float
        Suhu rata-rata selama musim tanam (°C)
    verbose : bool
        Tampilkan hasil detail (default: True)

    Returns:
    --------
    dict : Dictionary berisi hasil setiap layer dan prediksi akhir
    """

    x1 = soil_moisture
    x2 = soil_ph
    x3 = avg_temp

    # Parameter untuk setiap rule (dari soal UAS)
    params = {
        'R1': {'p': 0.0692, 'q': 0.0461, 'r': 0.00231},
        'R2': {'p': 0.0698, 'q': 0.0390, 'r': 0.00139},
        'R3': {'p': 0.0585, 'q': 0.0183, 'r': 0.00084},
        'R4': {'p': 0.0505, 'q': 0.0318, 'r': 0.00115}
    }

    # LAYER 1: FUZZIFIKASI
    # Menggunakan Gaussian membership function

    # Untuk soil_moisture (%)
    w1_moisture_low = gaussian_membership(x1, 40, 10)
    w1_moisture_high = gaussian_membership(x1, 70, 10)

    # Untuk soil_ph
    w1_ph_acidic = gaussian_membership(x2, 5.5, 0.5)
    w1_ph_neutral = gaussian_membership(x2, 7.0, 0.5)

    # Untuk avg_temp (°C)
    w1_temp_low = gaussian_membership(x3, 22, 3)
    w1_temp_high = gaussian_membership(x3, 28, 3)

    # LAYER 2: FIRING STRENGTH
    # AND operation menggunakan product (T-norm)
    w2_R1 = w1_moisture_low * w1_ph_acidic * w1_temp_low
    w2_R2 = w1_moisture_low * w1_ph_neutral * w1_temp_high
    w2_R3 = w1_moisture_high * w1_ph_acidic * w1_temp_high
    w2_R4 = w1_moisture_high * w1_ph_neutral * w1_temp_low

    # LAYER 3: NORMALISASI
    total_w2 = w2_R1 + w2_R2 + w2_R3 + w2_R4

    if total_w2 == 0:
        w3_R1 = w3_R2 = w3_R3 = w3_R4 = 0.25
    else:
        w3_R1 = w2_R1 / total_w2
        w3_R2 = w2_R2 / total_w2
        w3_R3 = w2_R3 / total_w2
        w3_R4 = w2_R4 / total_w2

    # LAYER 4: CONSEQUENT PARAMETERS
    # Sugeno linear model: fi = pi*x1 + qi*x2 + ri*x3
    f1 = params['R1']['p'] * x1 + params['R1']['q'] * x2 + params['R1']['r'] * x3
    f2 = params['R2']['p'] * x1 + params['R2']['q'] * x2 + params['R2']['r'] * x3
    f3 = params['R3']['p'] * x1 + params['R3']['q'] * x2 + params['R3']['r'] * x3
    f4 = params['R4']['p'] * x1 + params['R4']['q'] * x2 + params['R4']['r'] * x3

    w4_R1 = w3_R1 * f1
    w4_R2 = w3_R2 * f2
    w4_R3 = w3_R3 * f3
    w4_R4 = w3_R4 * f4

    # LAYER 5: OUTPUT AGREGASI
    output = w4_R1 + w4_R2 + w4_R3 + w4_R4

    hasil = {
        'input': {
            'soil_moisture': x1,
            'soil_ph': x2,
            'avg_temp': x3
        },
        'layer1': {
            'moisture_low': w1_moisture_low,
            'moisture_high': w1_moisture_high,
            'ph_acidic': w1_ph_acidic,
            'ph_neutral': w1_ph_neutral,
            'temp_low': w1_temp_low,
            'temp_high': w1_temp_high
        },
        'layer2': {
            'w_R1': w2_R1,
            'w_R2': w2_R2,
            'w_R3': w2_R3,
            'w_R4': w2_R4,
            'total': total_w2
        },
        'layer3': {
            'w_norm_R1': w3_R1,
            'w_norm_R2': w3_R2,
            'w_norm_R3': w3_R3,
            'w_norm_R4': w3_R4
        },
        'layer4': {
            'f1': f1,
            'f2': f2,
            'f3': f3,
            'f4': f4,
            'w_f1': w4_R1,
            'w_f2': w4_R2,
            'w_f3': w4_R3,
            'w_f4': w4_R4
        },
        'output': output,
        'parameters': params
    }

    if verbose:
        print_hasil_anfis(hasil)

    return hasil


def print_hasil_anfis(hasil):
    """Print hasil prediksi ANFIS dengan format yang rapi"""

    print("="*70)
    print("ANFIS - PREDIKSI PRODUKSI JAGUNG")
    print("="*70)
    print("\n1. DATA INPUT")
    print("-"*70)
    print(f"   Kelembaban Tanah (soil_moisture) : {hasil['input']['soil_moisture']}%")
    print(f"   pH Tanah (soil_pH)                : {hasil['input']['soil_ph']}")
    print(f"   Suhu Rata-rata (avg_temp)         : {hasil['input']['avg_temp']}°C")

    print("\n2. LAYER 1: FUZZIFIKASI (Gaussian Membership)")
    print("-"*70)
    print("   Soil Moisture:")
    print(f"     - Low  (μ=40, σ=10) : {hasil['layer1']['moisture_low']:.6f}")
    print(f"     - High (μ=70, σ=10) : {hasil['layer1']['moisture_high']:.6f}")
    print("\n   Soil pH:")
    print(f"     - Acidic  (μ=5.5, σ=0.5) : {hasil['layer1']['ph_acidic']:.6f}")
    print(f"     - Neutral (μ=7.0, σ=0.5) : {hasil['layer1']['ph_neutral']:.6f}")
    print("\n   Avg Temperature:")
    print(f"     - Low  (μ=22, σ=3) : {hasil['layer1']['temp_low']:.6f}")
    print(f"     - High (μ=28, σ=3) : {hasil['layer1']['temp_high']:.6f}")

    print("\n3. LAYER 2: FIRING STRENGTH (Product T-norm)")
    print("-"*70)
    print(f"   Rule 1 (Low & Acidic & Low)    : {hasil['layer2']['w_R1']:.6f}")
    print(f"   Rule 2 (Low & Neutral & High)  : {hasil['layer2']['w_R2']:.6f}")
    print(f"   Rule 3 (High & Acidic & High)  : {hasil['layer2']['w_R3']:.6f}")
    print(f"   Rule 4 (High & Neutral & Low)  : {hasil['layer2']['w_R4']:.6f}")

    print("\n4. LAYER 3: NORMALISASI")
    print("-"*70)
    print(f"   Total Firing Strength : {hasil['layer2']['total']:.6f}")
    print(f"   Normalized w1 (Rule 1) : {hasil['layer3']['w_norm_R1']:.6f}")
    print(f"   Normalized w2 (Rule 2) : {hasil['layer3']['w_norm_R2']:.6f}")
    print(f"   Normalized w3 (Rule 3) : {hasil['layer3']['w_norm_R3']:.6f}")
    print(f"   Normalized w4 (Rule 4) : {hasil['layer3']['w_norm_R4']:.6f}")

    print("\n5. LAYER 4: CONSEQUENT PARAMETERS (Sugeno Linear)")
    print("-"*70)
    p1, q1, r1 = hasil['parameters']['R1']['p'], hasil['parameters']['R1']['q'], hasil['parameters']['R1']['r']
    p2, q2, r2 = hasil['parameters']['R2']['p'], hasil['parameters']['R2']['q'], hasil['parameters']['R2']['r']
    p3, q3, r3 = hasil['parameters']['R3']['p'], hasil['parameters']['R3']['q'], hasil['parameters']['R3']['r']
    p4, q4, r4 = hasil['parameters']['R4']['p'], hasil['parameters']['R4']['q'], hasil['parameters']['R4']['r']

    print(f"   Rule 1: f1 = {p1:.4f}x1 + {q1:.4f}x2 + {r1:.5f}x3")
    print(f"          f1 = {hasil['layer4']['f1']:.6f}")
    print(f"          w̄1·f1 = {hasil['layer4']['w_f1']:.6f}")
    print()
    print(f"   Rule 2: f2 = {p2:.4f}x1 + {q2:.4f}x2 + {r2:.5f}x3")
    print(f"          f2 = {hasil['layer4']['f2']:.6f}")
    print(f"          w̄2·f2 = {hasil['layer4']['w_f2']:.6f}")
    print()
    print(f"   Rule 3: f3 = {p3:.4f}x1 + {q3:.4f}x2 + {r3:.5f}x3")
    print(f"          f3 = {hasil['layer4']['f3']:.6f}")
    print(f"          w̄3·f3 = {hasil['layer4']['w_f3']:.6f}")
    print()
    print(f"   Rule 4: f4 = {p4:.4f}x1 + {q4:.4f}x2 + {r4:.5f}x3")
    print(f"          f4 = {hasil['layer4']['f4']:.6f}")
    print(f"          w̄4·f4 = {hasil['layer4']['w_f4']:.6f}")

    print("\n6. LAYER 5: OUTPUT AGREGASI")
    print("-"*70)
    print(f"   Prediksi Produksi Jagung : {hasil['output']:.4f} ton/ha")

    print("\n7. INTERPRETASI")
    print("-"*70)
    if hasil['output'] < 3.0:
        status = "RENDAH"
        rekomendasi = "Perlu perbaikan kondisi tanah dan pengelolaan lahan."
    elif hasil['output'] < 5.0:
        status = "SEDANG"
        rekomendasi = "Produksi cukup baik, dapat ditingkatkan dengan optimasi pemupukan."
    else:
        status = "TINGGI"
        rekomendasi = "Produksi sangat baik, pertahankan kondisi optimal."

    print(f"   Status Produksi : {status}")
    print(f"   Rekomendasi     : {rekomendasi}")
    print("="*70)


if __name__ == "__main__":
    # Test dengan data dari soal UAS
    print("\nContoh Kasus dari Soal UAS:\n")
    print("Data: x1=60 (soil_moisture), x2=6.5 (soil_pH), x3=26 (avg_temp)")

    hasil = anfis_predict(
        soil_moisture=60,
        soil_ph=6.5,
        avg_temp=26
    )

    # Contoh kasus tambahan
    print("\n\nContoh Kasus Tambahan:\n")

    # Kasus 2: Kondisi Optimal
    print("\nKasus 2: Kondisi Optimal")
    anfis_predict(
        soil_moisture=70,
        soil_ph=7.0,
        avg_temp=28
    )

    # Kasus 3: Kondisi Kurang Baik
    print("\n\nKasus 3: Kondisi Kurang Baik")
    anfis_predict(
        soil_moisture=40,
        soil_ph=5.5,
        avg_temp=22
    )



Contoh Kasus dari Soal UAS:

Data: x1=60 (soil_moisture), x2=6.5 (soil_pH), x3=26 (avg_temp)
ANFIS - PREDIKSI PRODUKSI JAGUNG

1. DATA INPUT
----------------------------------------------------------------------
   Kelembaban Tanah (soil_moisture) : 60%
   pH Tanah (soil_pH)                : 6.5
   Suhu Rata-rata (avg_temp)         : 26°C

2. LAYER 1: FUZZIFIKASI (Gaussian Membership)
----------------------------------------------------------------------
   Soil Moisture:
     - Low  (μ=40, σ=10) : 0.135335
     - High (μ=70, σ=10) : 0.606531

   Soil pH:
     - Acidic  (μ=5.5, σ=0.5) : 0.135335
     - Neutral (μ=7.0, σ=0.5) : 0.606531

   Avg Temperature:
     - Low  (μ=22, σ=3) : 0.411112
     - High (μ=28, σ=3) : 0.800737

3. LAYER 2: FIRING STRENGTH (Product T-norm)
----------------------------------------------------------------------
   Rule 1 (Low & Acidic & Low)    : 0.007530
   Rule 2 (Low & Neutral & High)  : 0.065729
   Rule 3 (High & Acidic & High)  : 0.065729
   Rule 4 (H